In [112]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [113]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD

### 1. 讀入 MNSIT 數據集

In [114]:
from tensorflow.keras.datasets import mnist

In [115]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### 2. 資料整理

### Channel

CNN 要注意一張圖有多少個 channel, 開始我們因為只有灰階, 所以只有一個 channel。因此我們要轉一下我們的資料格式:

    (28,28) --> (28, 28, 1)

In [116]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255

In [117]:
x_test = x_test.reshape(10000, 28, 28, 1) / 255

In [118]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### 3. step 1: 打造函數學習機 (CNN)

In [119]:
model = Sequential()

In [120]:
model.add(Conv2D(16, (5,5), padding='same',
                input_shape=(28,28,1),
                activation='relu'))

In [121]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [122]:
model.add(Conv2D(32, (5,5), padding='same',
                activation='relu'))

In [123]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [124]:
model.add(Conv2D(64, (5,5), padding='same',
                activation='relu'))

In [125]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [126]:
model.add(Conv2D(128, (5,5), padding='same',
                activation='relu'))

In [127]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [128]:
model.add(Flatten())

In [129]:
model.add(Dense(80, activation='relu'))

In [130]:
model.add(Dense(10, activation='softmax'))

#### 看一下我們的神經網路

In [131]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 28, 28, 16)        416       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 14, 14, 32)        12832     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 7, 7, 64)          51264     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 3, 3, 64)          0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 3, 3, 128)        

#### 組裝

In [132]:
model.compile(loss='mse', optimizer=SGD(lr=0.087),
             metrics=['accuracy'])

### 4. step 2: fit

In [133]:
model.fit(x_train, y_train, batch_size=128, epochs=12)

Epoch 1/12
469/469 [==============================] - 6s 12ms/step - loss: 0.0899 - accuracy: 0.1629
Epoch 2/12
469/469 [==============================] - 5s 11ms/step - loss: 0.0898 - accuracy: 0.3116
Epoch 3/12
469/469 [==============================] - 5s 11ms/step - loss: 0.0894 - accuracy: 0.3460
Epoch 4/12
469/469 [==============================] - 5s 11ms/step - loss: 0.0884 - accuracy: 0.3416
Epoch 5/12
469/469 [==============================] - 5s 11ms/step - loss: 0.0755 - accuracy: 0.4390
Epoch 6/12
469/469 [==============================] - 5s 11ms/step - loss: 0.0351 - accuracy: 0.7617
Epoch 7/12
469/469 [==============================] - 5s 11ms/step - loss: 0.0190 - accuracy: 0.8769
Epoch 8/12
469/469 [==============================] - 5s 11ms/step - loss: 0.0138 - accuracy: 0.9110
Epoch 9/12
469/469 [==============================] - 5s 12ms/step - loss: 0.0108 - accuracy: 0.9313
Epoch 10/12
469/469 [==============================] - 5s 11ms/step - loss: 0.0091 - accura

### Step 3. 預測

In [139]:
result = model.predict_classes(x_test)

### 看看測試資料表現如何

In [140]:
loss, acc = model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0062 - accuracy: 0.9612


In [141]:
print(f'測試資料的正確率為 {acc*100:.2f}%')

測試資料的正確率為 96.12%


In [142]:
def my_predict(n):
    print('我可愛的 CNN 預測是', result[n])
    X = x_test[n].reshape(28,28)
    plt.imshow(X, cmap='Greys')

In [143]:
from ipywidgets import interact_manual

In [144]:
interact_manual(my_predict, n=(0, 9999));

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…

### 把我們的 model 存起來

In [145]:
model.save('myCNNmodel.h5')